In [1]:
!pip install advertools --quiet

In [2]:
import json
import config
gpt_response_file = config.gpt_response_lshs_file
gpt_filtered_rephrase_tweets_file = config.gpt_filtered_rephrase_lshs_file

In [3]:
out_file = open(gpt_response_file, "r")
gtp_dict = json.load(out_file)
out_file.close()

In [4]:
import re
import logging

logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S')
# The default levels are DEBUG, INFO, WARNING, ERROR, and CRITICAL.
print(logging.WARNING)
logging.getLogger()

30


<RootLogger root (INFO)>

In [5]:


def validateJSON(jsonData):
    try:
        json.loads(jsonData)
    except Exception as e:
        return False, e
    return True, None

def prepare_json(s, e, k):
    if (s[0] == '{' and s[1:].find('{')>-1):
        return fix_json(s, k)
    else:
        return convert_to_json(s, k)

def convert_to_json(text, k):
    logging.debug("not a json " + str(k))
    return text, 1
    
def get_line_and_col(input_string):
    # input_string = "Expecting ',' delimiter: line 3 column 56 (char 66)"

    # Define the regular expression pattern
    pattern = r'line (\d+) column (\d+) \(char \d+\)'  # Matches the line and column numbers

    # Use the search method to find the first match of the pattern in the string
    match = re.search(pattern, input_string)

    # Extract the line and column numbers from the match
    if match:
        line_number = int(match.group(1))
        column_number = int(match.group(2))
        return line_number, column_number
    
    return None, None

In [6]:
def extra_comma_before_list_or_dict_closes(fixed_json):
    pre_fixed_json = fixed_json
    fixed_json = re.sub(r'(,\s+)(]|})', r'\2', fixed_json, flags=re.DOTALL)
    if not pre_fixed_json == fixed_json:
        logging.debug('> extra_comma_before_list_or_dict_closes fixed')
        # print(fixed_json)
    return fixed_json

def missing_comma(fixed_json):
    pre_fixed_json = fixed_json
    fixed_json = re.sub(r'("])(\s+)(")', r'\1,\2\3', fixed_json, flags=re.DOTALL)
    if not pre_fixed_json == fixed_json:
        logging.debug('> missing common issue fixed')
        # print(fixed_json)
    return fixed_json

def fix_string_replace_single_quote_with_double(fixed_json):
    pre_fixed_json = fixed_json
    fixed_json = re.sub(r"(')(.+?)(')(\s*)(,|]|})", r'"\2"\4\5', fixed_json)
    if not pre_fixed_json == fixed_json:
        logging.debug('> fix_string_replace_single_quote_with_double')
        # print(fixed_json)
    return fixed_json

def add_quotation_around_key(fixed_json):
    # Fix the key without quotattions by adding "{key}"
    pre_fixed_json = fixed_json
    # ((,|{)[\t\n\r\s]+)([a-zA-Z0-9_ ]+)(\s*:) --> group 1 and 2 repeats
    fixed_json = re.sub(r'(,|{)([\t\n\r\s]+)([a-zA-Z0-9_ ]+)(\s*:)', r'\1\2"\3"\4', fixed_json, flags=re.DOTALL)
    
    if not pre_fixed_json == fixed_json:
        logging.debug('> key quotes issue fixed')
        # print(fixed_json)
    return fixed_json

def replace_consecutive_commas(fixed_json):
    #replace consecutive commas
    pre_fixed_json = fixed_json
    fixed_json = re.sub(r'(,)([\s\t\n])*(,)+', r'\1', fixed_json, flags=re.DOTALL) 
    if not pre_fixed_json == fixed_json:
        logging.debug('> consecutive_commas issue fixed')
        # print(fixed_json)
    return fixed_json


def fix_single_slash(fixed_json):
    #replace consecutive commas
    pre_fixed_json = fixed_json
    fixed_json = re.sub(r'(\w|\s)(\\)(\s+)', r'\1\2\2\3', fixed_json, flags=re.DOTALL) 
    if not pre_fixed_json == fixed_json:
        logging.debug('> single_slash issue fixed')
        # print(fixed_json)
    return fixed_json

def fix_square_brackets(fixed_json):
    pre_fixed_json = fixed_json
    # Fix the string data by adding square brackets around the "Problematic terms" values
    if not (re.search('("P|p)(roblematic)( |_)([\w\(\)]+":\s*)(\[)', fixed_json)): # bracket open is not found in the list
        # print('here1')
        
        fixed_json = re.sub(r'("P|p|R|r)(roblematic|ephrased)( |_)([\w\(\)]+":)[\s-]*(.*?)\s*((}|\s)*(,\n)|(}|\s)(}|"[\w\s]+":))', r'\1\2\3\4[\5]\6', fixed_json, flags=re.DOTALL)  # missing bracket "problematic_terms": "aa", "bb", (}) 975, 12900, 15940
        # fixed_json = re.sub(r'("P|p)(roblematic)( |_)([\w\(\)]+":\s*)(.*?)\s*((}|\s)*(,\n)|(}|\s)(}))', r'\1\2\3\4[\5]\6', fixed_json, flags=re.DOTALL)  # missing bracket "problematic_terms": "aa", "bb", (}) 975, 12900
        # fixed_json = re.sub(r'("P|p)(roblematic)( |_)([\w\(\)]+":\s*)(.*?)\s*(}|\s)*(,\n)', r'\1\2\3\4[\5]\6\7', fixed_json, flags=re.DOTALL)  # missing bracket "problematic_terms": "aa", "bb", 
    # if not (re.search('("P|p)(roblematic)( |_)([\w\(\)]+":\s*)(\[)', fixed_json)): # bracket open is not found in the list
    #     print('here2')
    #     fixed_json = re.sub(r'("P|p)(roblematic)( |_)([\w\(\)]+":\s*)(.*?)\s*(}|\s)(})', r'\1\2\3\4[\5]\6\7', fixed_json, flags=re.DOTALL)  # missing bracket "problematic_terms": "aa", "bb",(}) 
    ##     fixed_json = re.sub(r'("P|p)(roblematic)( |_)([\w\(\)]+":\s*)(.*?)\s*(\n})', r'\1\2\3\4[\5]\6', fixed_json, flags=re.DOTALL)  # missing bracket "problematic_terms": "aa", "bb"}
    
    fixed_json = re.sub(r'(")(,])(\s+)(")', r'\1],\3\4', fixed_json, flags=re.DOTALL)  # fix sitation forced by inserting bracket or may exist as error in json too
    
    fixed_json = re.sub(r'(")(\s*-\s*)(")', r'\1 , \3', fixed_json, flags=re.DOTALL) # list seperated by -
    
    if not pre_fixed_json == fixed_json:
        logging.debug('> square bracket issue fixed')
        # print(fixed_json)
    return fixed_json

def fix_escape_seq(fixed_json, k, max_iter=50):
    isValid = validateJSON(fixed_json)
    e = isValid[1]
    # print(k, e)
    for it in range(0, max_iter): # 50 errors to fix
        fixed=False
        e = str(e)
        l, c = get_line_and_col(e)
        # print(l,c)
        lines = fixed_json.split('\n')
        # print(lines[l-1])
        # print('-->', lines[l-1][c-3:c])
        bugs_fix = {'"': '\\"', ',': '",'}
        for i in range(min(len(lines[l-1])-1, c-2), max(0, c-4), -1):
            # print('-->', i, len(lines[l-1]))
            ch = lines[l-1][i]
            # print('--->', ch)
            if ch in bugs_fix:
                # print('yes')
                # print('<<', lines[l-1][:i] + bugs_fix[ch] + lines[l-1][i+1:])
                lines[l-1] = lines[l-1][:i] + bugs_fix[ch] + lines[l-1][i+1:]
                # print(lines[l-1])
                fixed = True
                break
        if not fixed:
            break
        fixed_json = '\n'.join(lines)
        
        isValid = validateJSON(fixed_json)
        if isValid[0]:
            logging.debug("Fixed " + str(k))
            return fixed_json, 0
        e = isValid[1]
        # print('loop')
    # Print the fixed JSON string
    logging.debug('Not fixed ' + str(k))
    # print(fixed_json)
    return fixed_json, 1
    
def fix_json(invalid_json, k):
    fixed_json = invalid_json
    
    function_list = [add_quotation_around_key,
                     replace_consecutive_commas, fix_square_brackets,
                     extra_comma_before_list_or_dict_closes, fix_single_slash, missing_comma,
                     fix_string_replace_single_quote_with_double                    
                    ]
    
    for func in function_list:
        fixed_json = func(fixed_json)
        isValid = validateJSON(fixed_json)
        if isValid[0]:
            logging.debug("Fixed " + str(k))
            return fixed_json, 0
        
    return fix_escape_seq(fixed_json, k)
    

def fix_bad_json():
    valid_tweets = {}
    cnt_broken = 0
    cnt_fixed = 0
    for k, v in gtp_dict.items():
        # too bad --> 9000 re behaviour is strange
        if int(k) in [9000, 9795, 13145, 13795, 14605]:
            cnt_broken += 1
            continue
        # if int(k)==14605 :
            # s=v
            # print(v)
            # fixed_json = v
        #     fix_square_brackets
        #    fixed_json = fix_square_brackets(fixed_json)
            # fixed_json = extra_comma_before_list_or_dict_closes(fixed_json)
            # print(fixed_json)
            # fixed_json = fix_string_replace_single_quote_with_double(fixed_json)
            # fixed_json, _ = fix_escape_seq(fixed_json, k)
        #     fix_escape_seq
            # print(fixed_json)
            # print(validateJSON(fixed_json))
            # break
        # else:
        #     continue
        isValid = validateJSON(v)
        if not isValid[0]:
            # d, c = prepare_json(v, isValid[1], k)
            try:
                # print(k)
                v, c = prepare_json(v, isValid[1], k)
                if c == 0:
                    cnt_fixed += 1
                else:
                    cnt_broken += 1
                    continue
                # json.loads(d)
            except Exception as e:
                # cnt_broken+= 1
                print('not fixable', k, e)
                if (str(e).find("Expecting")>-1):

                    pass
                continue
        
        valid_tweets[k] = v
    logging.info('Total Fixed:' + str(cnt_fixed*5))
    logging.info('Total Broken:' + str(cnt_broken*5))
    logging.info('Total Entries: ' + str(len(gtp_dict.keys())*5))
    logging.info('Total Recovered Entries: ' + str(len(valid_tweets.keys())*5))
    p = str(round((len(valid_tweets.keys())*5)/(len(gtp_dict.keys())*5)*100, 2)) +'%'
    logging.info('Percentage of Total Valid: ' + p)
    return valid_tweets
    


valid_tweets = fix_bad_json()

2023-06-11 20:17:04 INFO     Total Fixed:330
2023-06-11 20:17:04 INFO     Total Broken:1805
2023-06-11 20:17:04 INFO     Total Entries: 10790
2023-06-11 20:17:04 INFO     Total Recovered Entries: 8985
2023-06-11 20:17:04 INFO     Percentage of Total Valid: 83.27%


In [7]:
import re
import advertools as adv

rephrase_keys = ['rephrased_tweets', 'Rephrased versions', 'rephrased', 'rephrased_versions',
                     'rephrased versions', 'Rephrased Versions', 'rephrasedVersions', 'inclusive_versions',
                     'Inclusive rephrased versions', 'rephrasedTweet', 'rephrasedTweets', 'inclusive_rephrased_tweets'
                    ]
rephrase_count = ['rephrased_tweet_', 'rephrased_', 'rephrased', 'Rephrased', 'Rephrased ', 'Rephrased tweet ', 
                  'rephrased_version_', 'rephrased_tweet', 'Rephrased Tweet ', 'inclusive_tweet_', 
                  'Rephrased Version ']
counter = ['1', '2', '3']

def fix_keys(j):
    fixed = {}
    for k in j.keys():
        k_fixed = re.findall(r'\d+', k)[0]
        fixed[k_fixed] = j[k]
    return fixed

def __rephrase_tweet_dict__(i, t_info, offset):
    out = None
    found = False
    for r in rephrase_keys:
        if r in t_info:
            out = t_info[r]
            if type(out)==str:
                out = [out]
            if type(out[0]) == dict: # multilevel
                out = [out[i]['tweet'] for i in range(0, len(out))]
            found = True
            break

    if not found:
        l = []
        for r in rephrase_count:
            for c in counter:
                rc = r+c
                if rc in t_info:
                    l.append(t_info[rc])
                    found = True
        if found:
            out = l

    if not found:
        logging.warning('Tweets Rephrase keys not able to extract ' + str(t_info.keys()) + ' from '  + str(i+offset))
    return out

def __rephrased_tweet_str_followed_by_digit__(t_info):
    out = None
    l = []
    found = False
    for r in rephrase_keys + ['Inclusive Rephrasing', 'Rephrased Tweet', 'Rephrased Version', 'Revised Tweet', 'Inclusive Rephrase',
                            'Inclusive language rephrased version', 'Inclusive version', 'Inclusive rephrased tweet', 
                             ]:
        # followed by digit
        for e in t_info:
            if type(e) == str:
                s = re.search(r'(' + r + '\s*)(\d:\s*)', e, re.IGNORECASE)
                if not s is None and s.span()[0]==0:
                    # print('yes', e[0:s.span()[1]])
                    l.append(e[s.span()[1]:])
                    found = True
        if l:
            out=l
            break
    return out, found

def __safe__(v):
    if v.lower()=='none':
        return ''
    else:
        return v

def __rephrased_tweet_str_not_followed_by_digit__(t_info):
    out = None
    l = []
    found = False
    break_now = False
    for r in set(map(str.lower, rephrase_keys+ ['Rephrased versions', 'Alternative rephrased tweet', 'Alternative', 'Rephrased tweets', 
                             'Rephrased tweet', 'Rephrased tweet\(s\)', 'Rephrased tweets', 'Inclusive Language', 
                             'Inclusive rephrased tweets', 'Inclusive versions', 'Rephrased tweet options', 
                             'Inclusive rephrased version', 'Inclusive language tweet', 'Inclusive language alternatives',
                             '[\w\s]*Rephrased version'])):
        # not followed by a digit and can take other terms
        for n, e in enumerate(t_info):
            if type(e) == str:
                s = re.search(r'(.*' + r + '\s*)(:\s*)', e, re.IGNORECASE)
                if not s is None and s.span()[0]==0:
                    t = e[s.span()[1]:]
                    if len(t.strip()):
                        l.append(__safe__(t))
                        # print(t, t_info)
                        found = True
                    elif n==0 or n>0:  # match lst 0 matches rephrase key, and following contains items contains rephrased versions
                        for i in range(n+1, len(t_info)):
                            if t_info[i].strip()[0] == '-':
                                l.append(t_info[i].strip()[1:])
                                found = True
                            elif re.search(r"(^\s*\d\.)(.*)", t_info[i]): # check 1. tweet
                                l.append(re.search(r"(^\s*\d\.)(.*)", t_info[i]).group(2))
                                found = True
                            elif (re.findall("@([a-zA-Z0-9_]{1,50})", t_info[i]) or re.findall("#([a-zA-Z0-9_]{1,50})", t_info[i]) 
                                  or len(t_info[i].split(' '))>5 or adv.extract_emoji(t_info[i])) : # check if text is a tweet through ht, men, emoji or more than 5 words
                                l.append(t_info[i])
                                found = True
                    if found:
                        # 
                        break_now = True
                        # if n==2:
                        #     print(id, t_info, l)
                        break
        if break_now:
            break

    if l:
        out=l
        # print(id, l, t_info)
    return out, found

def __rephrase_tweet_list__(id, t_info, offset):
    out = None
    found = False
    if len(t_info) > 0 and type(t_info[0])==dict:
        # l = []
        for i in range(0, len(t_info)):
            for r in rephrase_keys + ['rephrased_tweet', 'tweet']:
                if r in t_info[i]:
                    # print('--', t_info[i][r])
                    # l.append(t_info[i][r])
                    out = t_info[i][r]
                    found = True
                    break
        # if found:
        #     out = l
    elif len(t_info)==0:
        # print(id)
        out = []
    elif len(t_info) > 1:
        l = []
        out, found = __rephrased_tweet_str_followed_by_digit__(t_info)
        if not found:
            out, found = __rephrased_tweet_str_not_followed_by_digit__(t_info)
        if not found:
            if len(t_info):
                1
                # print(t_info, id)
                # for e in t_info:
                #     if e.find('1:')>-1:
                #         print(t_info)
                #         break
    return out

def get_rephrase_tweets(i, t_info, offset):
    out = None
    if type(t_info)==dict:
        out = __rephrase_tweet_dict__(i, t_info, offset)
    elif (type(t_info)==list):
        out = __rephrase_tweet_list__(i, t_info, offset)
    else:
        logging.warning('Ignoring GPT Tweet of ' + str(type(t_info)) + ' type from ' + str(i+offset))
    return out


def get_flattened_list(nested_list):
    lst = []
    if type(nested_list) == str:
        return [nested_list]
    for l in nested_list:
        if type(l)==list:
            lst.extend(l)
        else:
            lst.append(l)
    return lst


def check_duplicates(out_tweet, start_id, bunch_size):
    z = [out_tweet[i] for i in range(start_id, start_id + bunch_size) if i in out_tweet]
    if len(get_flattened_list(z)) == len(set(get_flattened_list(z))):
        return out_tweet
    else:
        for i in range(start_id, start_id + bunch_size):
            out_tweet[i] = ''
    logging.warning('Tweets contain some duplications, poor quality dropping from ' + str(start_id) + ' to '+ str(start_id + bunch_size))
    return out_tweet      
    

def extract_json(it, bunch_size, out_tweet):
    start_id = int(it[0])
    # if not start_id in [6995]:#30, 13240]: #[19910, 240, 13405, 0, 1245]:
    #     return out_tweet
    j = json.loads(it[1])

    # print(start_id, len(j))
    
    if bunch_size == len(j):
        # return out_tweet
        if type(j)==dict:
            j = fix_keys(j)
            # print(j.keys())
            # print(str(start_id) in j)
            if(str(start_id) in j):
                offset = 0
            else:
                offset = start_id - 1
            for i in range(start_id, start_id + bunch_size):
                t_info = j[str(i - offset)]
                out = get_rephrase_tweets(i, t_info, offset)
                # print('---', out)
                if not out is None:
                    out_tweet[i] = get_flattened_list(out)
                else:
                    # print(j)
                    # print('dict', i)
                    logging.warning('Cannot extract from dictionary from ' + str(i))
            out_tweet = check_duplicates(out_tweet, start_id, bunch_size)
        else:
            # print(type(j), start_id)
            # print(len(j))
            for i in range(0, bunch_size):
                t_info = j[i]
                out = get_rephrase_tweets(i, t_info, start_id)
                if not out is None:
                    out_tweet[start_id+i] = out
                else:
                    print('list', start_id+i)
    elif type(j)==dict:
        # return out_tweet
        if 'tweets' in j and len(j['tweets']) == bunch_size:
            # return out_tweet
            # print(start_id, len(j['tweets']))
            j = j['tweets']
            for i in range(0, bunch_size):
                t_info = j[i]
                out = get_rephrase_tweets(i, t_info, start_id)
                if not out is None:
                    out_tweet[start_id+i] = out
                else:
                    pass
                    # print('out list', start_id+i, t_info)
        else:
            out_tweet = extract_from_diverse_dict(j, start_id, bunch_size, out_tweet)
    else:
        print('Non Json type', j)
                
    return out_tweet

def extract_from_diverse_dict(j, start_id, bunch_size, out_tweet):
    # print(start_id, j.keys())
    for i in range(start_id, start_id + bunch_size):
        found = False
        keys = [str(i), str(i) + '.']
        for k in keys:
            if k in j:
                t_info = j[k]
                if type(t_info)==list and len(t_info)==3:
                    out = t_info
                    found = True
                else:
                    out, found = __rephrased_tweet_str_followed_by_digit__(t_info)
                if found:
                    out_tweet[i] = out
                    break
    
    if not found:
        wrong = False
        for i in range(start_id, start_id + bunch_size):
            found = False
            for k in j.keys():
                for r in set(rephrase_keys + ['Rephrased tweets', 'tweet_rephrased', 'tweets_rephrased']):
                    # print('(' + str(i) + '.*' + r + ')|(' + r + '.*' + str(i) +')')
                    # pattern = r'(' + str(i) + '.*' + r + ')|(' + r + '.*' + str(i) +')|(^' + r + '$)'
                    pattern_without_tweet_id = r'(^' + r + '$)'
                    pattern_with_tweet_id = r'(' + str(i) + '.*' + r + ')|(' + r + '.*' + str(i) +')'
                    if re.search(pattern_without_tweet_id, k, re.IGNORECASE):
                        if i in out_tweet and type(j[k])==str:
                            out_tweet[i].append(j[k])
                        elif type(j[k])==str:
                            out_tweet[i] = [j[k]]
                        else:
                            # print(i)
                            # print(k, pattern_without_tweet_id)
                            v = multi_level_rephrased(j[k], i-start_id, i, with_digit=False)
                            if v:
                                out_tweet[i] = v
                            else:
                                wrong = True
                                break
                        found = True
                        break
                    if re.search(pattern_with_tweet_id, k, re.IGNORECASE):
                        if i in out_tweet and type(j[k])==str:
                            out_tweet[i].append(j[k])
                        elif type(j[k])==str:
                            out_tweet[i] = [j[k]]
                        else:
                            # print(i)
                            # print(k, pattern_with_tweet_id)
                            v = multi_level_rephrased(j[k], i-start_id, i, with_digit=True)
                            if v:
                                out_tweet[i] = v
                            else:
                                wrong = True
                                break
                        found = True
                        break
            if not found and not wrong:
                for r in rephrase_keys:
                    keys = [str(i) + '. ' + r, r]
                    for k in keys:
                        if k in j:
                            out_tweet[i] = j[k]
                            found = True
                            break
                        if found:
                            break
            if not found or not wrong:
                # print(i, 'not found', j.keys())
                if wrong:
                    logging.warning('Not well formed Json in the diverse dictionary with' + str(j.keys()))
                else:
                    logging.warning('Not found in diverse dictionary with' + str(j.keys()))
    return out_tweet

def multi_level_rephrased(v, i, idx, with_digit=True):
    lst = []
    if type(v)==list:
        if type(v[0]) == str:
            s1 = re.search('^(\d+).\s*(.*)$', v[0], re.IGNORECASE)
            if s1:
                l = []
                for cnt in range(0, len(v)):
                    s1 = re.search('^(\d+).\s*(.*)$', v[cnt], re.IGNORECASE)
                    if str(idx) == s1.group(1):
                        l.append(s1.group(2))
                # print(l)
                # print(idx, s1.group(1))
                return l
            else:
                if with_digit:
                    # print(with_digit, v, idx)
                    return v
                else:
                    if not len(v) == 5:
                        return None
                    # print(with_digit, v[i], idx)
                    if type(v[i])==str:
                        return [v[i]]
                    else:
                        return v[i]
                
#                 if len(v) == 1:
#                     return v
#                 else:
                    
#                     return v[i]
            
        else:
            for r in set(rephrase_keys + ['tweet', 'text', 'rephrased_version', 'rephrased_tweet']):
                if r in v[i]:
                    out = v[i][r]
                    if type(out)==str:
                        out = [out]
                    return out
            # try:
            #     # for r in set(rephrase_keys + ['tweet', 'text']):
            #     #     return [v[i][r]]
            #     for r in set(rephrase_keys + ['tweet', 'text', 'rephrased_version', 'rephrased_tweet']):
            #         if r in v[i]:
            #             return [v[i][r]]
            #     1/0
            # except:
            #     print(idx)
            #     print(v[i])
            #     for r in set(['tweet']):
            #         return [v[i][r]]
    return None
        
    

def arrange_tweets(bunch_size=5):
    out_tweet = {}
    for it in valid_tweets.items():
        out_tweet = extract_json(it, bunch_size, out_tweet)
    p = str(round((len(out_tweet.keys()))/(len(valid_tweets.keys())*5)*100, 2)) +'%'
    logging.info('Percentage of GPT Rephrase of Tweets: ' + p)
    return out_tweet


rephrase_tweet_gpt = arrange_tweets(bunch_size=5)

2023-06-11 20:17:05 WARNING  Tweets contain some duplications, poor quality dropping from 150 to 155
2023-06-11 20:17:05 WARNING  Tweets contain some duplications, poor quality dropping from 335 to 340
2023-06-11 20:17:05 WARNING  Tweets contain some duplications, poor quality dropping from 370 to 375
2023-06-11 20:17:05 WARNING  Tweets contain some duplications, poor quality dropping from 510 to 515
2023-06-11 20:17:05 WARNING  Cannot extract from dictionary from 630
2023-06-11 20:17:05 WARNING  Cannot extract from dictionary from 631
2023-06-11 20:17:05 WARNING  Cannot extract from dictionary from 632
2023-06-11 20:17:05 WARNING  Cannot extract from dictionary from 633
2023-06-11 20:17:05 WARNING  Cannot extract from dictionary from 634
2023-06-11 20:17:05 WARNING  Tweets contain some duplications, poor quality dropping from 650 to 655
2023-06-11 20:17:05 WARNING  Not found in diverse dictionary withdict_keys(['765_original_tweet', '765_rephrased_tweets', '765_suspected_problematic_t

In [8]:
# display(rephrase_tweet_gpt[16710])
# print(valid_tweets['16710'])
# display(rephrase_tweet_gpt[0])

In [9]:
# display(rephrase_tweet_gpt[17480])
# print(valid_tweets['17480'])
# display(rephrase_tweet_gpt[0])

In [10]:
# display(rephrase_tweet_gpt[20115])
# print(valid_tweets['20115'])

In [11]:
len(rephrase_tweet_gpt)

8830

In [12]:
len(valid_tweets)*5, max(map(int, valid_tweets.keys()))

(8985, 10780)

In [13]:
# for k in map(int, valid_tweets.keys()):
#     if not k in rephrase_tweet_gpt:
#         print(k)

In [14]:
# offensive derogatory <-- check and also empty ones
# search for Suspected problematic term:
# 140 only extracts one rephrased version

In [15]:
def contains_filtered_words(lst):
    # filter_list = ['offensive', 'derogatory', 'problematic terms*']
    # filter_list = ['offensive']
    filter_list = ['problematic terms*']
    for f in filter_list:
        pattern = '\\b' + f +'\\b'
        # print(pattern)
        for e in lst:
            if re.search(pattern, e, re.IGNORECASE):
                return True
    return False
    

contains_filtered_words(['This person is arguing with my dad. [problematic term: bitchh]', "If you associate with someone who is not cool, you are not cool either."])

True

In [16]:
filtered_rephrase_tweet_gpt = {}
def error_stats():
    
    error = {'empty': 0, 'filtered': 0} 
    for i, l in rephrase_tweet_gpt.items():
        try:
            if len(l) ==0 or len(''.join(l))==0:
                # print('-->', i, l)
                error['empty'] += 1
                1
            elif contains_filtered_words(l):
                # print(i, 'ignore', l)
                error['filtered'] += 1
                1
            # elif len(l)==4:
            #     print(i, l)
            #     1
            else:
                filtered_rephrase_tweet_gpt[i] = l
        except:
            pass
            # print(i, l)

    out_file = open(gpt_filtered_rephrase_tweets_file, "w")
    json.dump(filtered_rephrase_tweet_gpt, out_file, indent = 2)
    out_file.close()

    logging.info('Error Stats: ' + str(error))
    p = str(round(len(filtered_rephrase_tweet_gpt)/ (len(valid_tweets)*5 )*100, 2)) +'%'
    logging.info('Percentage of Filtered GPT Rephrase of Tweets: ' + p)
    
error_stats()

2023-06-11 20:17:23 INFO     Error Stats: {'empty': 517, 'filtered': 178}
2023-06-11 20:17:23 INFO     Percentage of Filtered GPT Rephrase of Tweets: 90.54%


In [17]:
print(len(filtered_rephrase_tweet_gpt))

8135


In [ ]:
8135/10780